In [ ]:
import os
from pathlib import Path

import matplotlib as mpl
import numpy as np
import pandas as pd
import xarray as xr

from oslofjord_sim.stuff import (
    add_river_coordinates,
    fill_nans_with_nearest,
    get_river_indices,
    reformat_river_dataset,
    regrid_from_norkyst,
)

### Regridding
Open the downloaded datasets and regrid them to the coordinates from a bathymetry file.

In [ ]:
FILES = sorted(
    os.path.join(Path.home(), "FjordSim_data", "oslofjord", f)
    for f in os.listdir(os.path.join(Path.home(), "FjordSim_data", "oslofjord"))
    if f.startswith("NorKyst-800m_ZDEPTHS_avg_")
)

In [ ]:
ds_grid = xr.open_dataset(os.path.join(Path.home(), "FjordSim_data", "oslofjord", "bathymetry_105to232.nc"))

In [ ]:
lat_diff = np.diff(ds_grid["lat"].values)[0]
lat_faces = np.zeros(shape=ds_grid["lat"].values.shape[0] + 1)
lat_faces[0] = ds_grid["lat"].values[0] - lat_diff / 2
lat_faces[1:] = ds_grid["lat"].values + lat_diff / 2

In [ ]:
lon_diff = np.diff(ds_grid["lon"].values)[0]
lon_faces = np.zeros(shape=ds_grid["lon"].values.shape[0] + 1)
lon_faces[0] = ds_grid["lon"].values[0] - lon_diff / 2
lon_faces[1:] = ds_grid["lon"].values + lon_diff / 2

In [ ]:
z_faces = ds_grid.z_faces.values
z_centers = [(z_faces[i] + z_faces[i + 1]) / 2 for i in range(len(z_faces) - 1)]
ds_out_c = xr.Dataset(
    {
        "lat": (["lat"], ds_grid["lat"].values, {"units": "degrees_north"}),
        "lon": (["lon"], ds_grid["lon"].values, {"units": "degrees_east"}),
    }
)
ds_out_u = xr.Dataset(
    {
        "lat": (["lat"], ds_grid["lat"].values, {"units": "degrees_north"}),
        "lon": (["lon"], lon_faces, {"units": "degrees_east"}),
    }
)
ds_out_v = xr.Dataset(
    {
        "lat": (["lat"], lat_faces, {"units": "degrees_north"}),
        "lon": (["lon"], ds_grid["lon"].values, {"units": "degrees_east"}),
    }
)

In [ ]:
time_list = []
temp_list = []
salt_list = []
u_list = []
v_list = []

regridder_rho, regridder_u, regridder_v = None, None, None
for file in FILES:
    ds_in = xr.open_dataset(file)
    new_time = pd.date_range(start=ds_in.time[0].values, end=ds_in.time[-1].values, freq="D")
    ds_in = ds_in.reindex(time=new_time, method="ffill")
    regridder_rho, regridder_u, regridder_v, np_time, np_temp, np_salt, np_u, np_v = regrid_from_norkyst(
        regridder_rho, regridder_u, regridder_v, ds_in, ds_out_c, ds_out_u, ds_out_v, np.array(z_centers)
    )
    time_list.append(np_time)
    temp_list.append(np_temp)
    salt_list.append(np_salt)
    u_list.append(np_u)
    v_list.append(np_v)
    print(f"File {file} processed.")

In [ ]:
np_time = np.concatenate(time_list, axis=0)
np_temp = np.concatenate(temp_list, axis=0).astype(np.float32)
np_salt = np.concatenate(salt_list, axis=0).astype(np.float32)
np_u = np.concatenate(u_list, axis=0).astype(np.float32)
np_v = np.concatenate(v_list, axis=0).astype(np.float32)

In [ ]:
Tout_lambda = np.zeros_like(np_temp)
Sout_lambda = np.zeros_like(np_salt)
Uout_lambda = np.zeros_like(np_u)
Vout_lambda = np.zeros_like(np_v)

In [ ]:
dsout = xr.Dataset(
    {
        "T": (["time", "Nz", "Ny", "Nx"], np_temp),
        "T_lambda": (["time", "Nz", "Ny", "Nx"], Tout_lambda),
        "S": (["time", "Nz", "Ny", "Nx"], np_salt),
        "S_lambda": (["time", "Nz", "Ny", "Nx"], Sout_lambda),
        "u": (["time", "Nz", "Ny", "Nx_faces"], np_u),
        "u_lambda": (["time", "Nz", "Ny", "Nx_faces"], Uout_lambda),
        "v": (["time", "Nz", "Ny_faces", "Nx"], np_v),
        "v_lambda": (["time", "Nz", "Ny_faces", "Nx"], Vout_lambda),
    },
    coords={
        "time": np_time,
        "Nz": z_centers,
        "Nz_faces": z_faces,
        "Ny": ds_grid["lat"].values,
        "Ny_faces": lat_faces,
        "Nx": ds_grid["lon"].values,
        "Nx_faces": lon_faces,
    },
)

In [ ]:
dsout

In [ ]:
z_level = -1

In [ ]:
dsout.T.isel(time=0, Nz=z_level).plot()

In [ ]:
dsout.S.isel(time=0, Nz=z_level).plot()

In [ ]:
dsout.u.isel(time=0, Nz=z_level).plot()

In [ ]:
dsout.v.isel(time=0, Nz=z_level).plot()

### Prepare the forcing dataset

In [ ]:
ds = dsout.copy(deep=True)

In [ ]:
np_z_centers = (ds_grid.z_faces.values[:-1] + ds_grid.z_faces.values[1:]) / 2
oy, ox, oz = (value for value in ds_grid.sizes.values())
np_mask = (np_z_centers > ds_grid.h.values[..., np.newaxis]).astype(int).transpose(2, 0, 1)

In [ ]:
np_mask_u = np.zeros((oz - 1, oy, ox + 1))
np_mask_u[:, :, :-1] = np_mask
np_mask_u[:, :, 1:] = np.where(np_mask_u[:, :, 1:] == 0, np_mask, np_mask_u[:, :, 1:])

In [ ]:
np_mask_v = np.zeros((oz - 1, oy + 1, ox))
np_mask_v[:, :-1, :] = np_mask
np_mask_v[:, 1:, :] = np.where(np_mask_v[:, 1:, :] == 0, np_mask, np_mask_v[:, 1:, :])

In [ ]:
ds.T.values = fill_nans_with_nearest(ds.T) * np_mask
ds.S.values = fill_nans_with_nearest(ds.S) * np_mask
ds.u.values = fill_nans_with_nearest(ds.u) * np_mask_u
ds.v.values = fill_nans_with_nearest(ds.v) * np_mask_v

In [ ]:
ds.v.isel(time=0, Nz=-1).plot()

Check once again for data gaps in the downloaded datasets

In [ ]:
ds = ds.sortby("time")

In [ ]:
ds

In [ ]:
new_time = pd.date_range(start=ds.time[0].values, end=ds.time[-1].values, freq="D")
new_time[~new_time.isin(ds.time.values)]  # this shows which dates are missing in the original data

In [ ]:
ds = ds.reindex(time=new_time, method="ffill")

Fill in lambdas for the sourthern boundary conditions.
Lambdas that -1 < lambda < 1 are relaxation rates.
Lambdas > 2 mean flux in x direction (west to east).
Lambdas < -1 mean flux in y direction (south to north).

In [ ]:
southern_edge = 10  # use relaxation at 10 southernmost 'layers' at all depths
lambda_relaxation = 1 / (60 * 60 * 24)

In [ ]:
ds["NUT"] = (["time", "Nz", "Ny", "Nx"], np.zeros_like(ds.T.values))
ds["NUT_lambda"] = (["time", "Nz", "Ny", "Nx"], np.zeros_like(ds.T_lambda.values))
ds["C"] = (["time", "Nz", "Ny", "Nx"], np.zeros_like(ds.T.values))
ds["C_lambda"] = (["time", "Nz", "Ny", "Nx"], np.zeros_like(ds.T_lambda.values))

In [ ]:
ds["T_lambda"][:, :, :southern_edge, :] = (lambda_relaxation) * np_mask[:, :southern_edge, :]
ds["T_lambda"] = ds["T_lambda"].astype(np.float32)
ds["S_lambda"][:, :, :southern_edge, :] = (lambda_relaxation) * np_mask[:, :southern_edge, :]
ds["S_lambda"] = ds["S_lambda"].astype(np.float32)
ds["u_lambda"][:, :, :southern_edge, :] = (lambda_relaxation) * np_mask_u[:, :southern_edge, :]
ds["u_lambda"] = ds["u_lambda"].astype(np.float32)
ds["v_lambda"][:, :, :southern_edge, :] = (lambda_relaxation) * np_mask_v[:, :southern_edge, :]
ds["v_lambda"] = ds["v_lambda"].astype(np.float32)

In [ ]:
ds["T"][:, :, :, :] = np.where(np_mask == 0, np.nan, np_mask) * ds["T"][:, :, :, :]
ds["S"][:, :, :, :] = np.where(np_mask == 0, np.nan, np_mask) * ds["S"][:, :, :, :]
ds["NUT"][:, :, :, :] = np.where(np_mask == 0, np.nan, np_mask) * ds["NUT"][:, :, :, :]
ds["C"][:, :, :, :] = np.where(np_mask == 0, np.nan, np_mask) * ds["C"][:, :, :, :]
ds["u"][:, :, :, :] = np.where(np_mask_u == 0, np.nan, np_mask_u) * ds["u"][:, :, :, :]
ds["v"][:, :, :, :] = np.where(np_mask_v == 0, np.nan, np_mask_v) * ds["v"][:, :, :, :]

In [ ]:
ds

### Add rivers

In [ ]:
ds_rivers = xr.open_dataset(os.path.join(Path.home(), "FjordSim_data", "oslofjord", "of800_rivers_v9_1990_2022_RA1.nc"))
ds_rivers = ds_rivers.isel(s_rho=-1)  # all layers data is the same, keep only 1
ds_rivers = ds_rivers.assign_coords(river=np.arange(1, 26))

Refactor a rivers dataset for convenience

In [ ]:
ds_rivers = reformat_river_dataset(ds_rivers)

Add rivers coordinates from another file

In [ ]:
df_rivers = pd.read_csv(os.path.join(Path.home(), "FjordSim_data", "oslofjord", "OF800_rivers.csv"))
ds_rivers = add_river_coordinates(ds_rivers, df_rivers)

In [ ]:
surface_mask = np_mask[-1, :, :]

Fill in river_indices with valid cell y, x coordinates

In [ ]:
river_indices = get_river_indices(ds, ds_rivers, surface_mask)

Put rivers into the forcing dataset

In [ ]:
temperature_idx = list(ds_rivers.properties).index("river_temp")
nitrate_idx = list(ds_rivers.properties).index("river_N3_n")
for river_num, (y, x) in river_indices.items():
    ds_river = ds_rivers.sel(river_number=river_num)
    temperature = ds_river.rivers.isel(properties=temperature_idx).sel(river_time=slice("2020-01-01", "2020-12-31"))
    ds["T"].values[:, -1, y, x] = temperature.values
    ds["T"][:, :, :, :] = np.where(np_mask == 0, np.nan, np_mask) * ds["T"][:, :, :, :]
    ds["T_lambda"].values[:, -1, y, x] = 1 / (60 * 60)
    ds["S"].values[:, -1, y, x] = 0
    ds["S"][:, :, :, :] = np.where(np_mask == 0, np.nan, np_mask) * ds["S"][:, :, :, :]
    ds["S_lambda"].values[:, -1, y, x] = 1 / (60 * 60)
    nitrate = ds_river.rivers.isel(properties=nitrate_idx).sel(river_time=slice("2020-01-01", "2020-12-31"))
    ds["NUT"].values[:, -1, y, x] = nitrate.values
    ds["NUT"][:, :, :, :] = np.where(np_mask == 0, np.nan, np_mask) * ds["NUT"][:, :, :, :]
    ds["NUT_lambda"].values[:, -1, y, x] = 1 / (60 * 60)

In [ ]:
ds.S.isel(time=-1, Nz=-1, Ny=slice(180, 190), Nx=slice(5, 15)).plot(cmap=mpl.colormaps.get_cmap("Spectral"))

In [ ]:
ds.S.isel(time=-1, Nz=-1, Ny=slice(215, 225), Nx=slice(40, 50)).plot(cmap=mpl.colormaps.get_cmap("Spectral"))

In [ ]:
ds.S_lambda.isel(time=-1, Nz=-1).where(lambda x: x != 0).plot(cmap=mpl.colormaps.get_cmap("Spectral"), figsize=(7, 10))

In [ ]:
ds.S.isel(time=-1, Nz=-1).plot(cmap=mpl.colormaps.get_cmap("Spectral"), figsize=(7, 10))

In [ ]:
ds["C"][:, 11, 185, 10] = 1.0
ds["C_lambda"].values[:, 11, 185, 10] = 1 / (60 * 60)

In [ ]:
ds["C"][:, 12, 220, 45] = 1.0
ds["C_lambda"].values[:, 12, 220, 45] = 1 / (60 * 60)

In [ ]:
ds.C.isel(time=-1, Nz=12).plot(cmap=mpl.colormaps.get_cmap("Spectral"), figsize=(7, 10))

In [ ]:
for var in ds.data_vars:
    ds[var] = ds[var].astype(np.float32)

In [ ]:
ds

### Save

In [ ]:
encoding = {var: {"zlib": True, "complevel": 5} for var in ds.data_vars}
ds.to_netcdf(
    os.path.join(Path.home(), "FjordSim_data", "oslofjord", "forcing_105to232.nc"),
    encoding=encoding,
)